# ✅ Verified Hybrid Forecasting Notebook with Results Output

In [ ]:
!pip install prophet tensorflow seaborn openpyxl


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import os

SAVE_DIR = "hybrid_prophet_lstm_output"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
df = pd.read_excel("usask.sec.min_short_v2.xlsx")
df.columns = ['minute', 'requests']
df['minute'] = pd.to_datetime(df['minute'], unit='m', origin='unix')
df.rename(columns={'minute': 'ds', 'requests': 'y'}, inplace=True)


In [ ]:
windows = [-90, -75, -60, -45, -30]
horizons = [1, 5, 10, 15]
window_size = 30
results = []

for win in windows:
    hist = df[:win]
    test_full = df[win:]

    model = Prophet(
        growth='linear',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10.0,
        holidays_prior_scale=10.0,
        n_changepoints=25
    )
    model.add_seasonality(name='hourly', period=1, fourier_order=3)
    model.fit(hist)

    future = model.make_future_dataframe(periods=len(test_full), freq='min')
    forecast = model.predict(future)
    hist = hist.copy()
    hist['yhat'] = forecast.loc[:len(hist)-1, 'yhat'].values
    hist['residual'] = hist['y'] - hist['yhat']

    residual_series = hist['residual'].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    res_scaled = scaler.fit_transform(residual_series)

    X, y_lstm = [], []
    for i in range(window_size, len(res_scaled) - max(horizons)):
        X.append(res_scaled[i - window_size:i])
        y_lstm.append(res_scaled[i:i + max(horizons)].flatten())
    X, y_lstm = np.array(X), np.array(y_lstm)

    best_loss = float('inf')

    for units in [32, 64]:
        model_lstm = Sequential()
        model_lstm.add(LSTM(units, activation='tanh', input_shape=(X.shape[1], X.shape[2])))
        model_lstm.add(Dense(max(horizons)))
        model_lstm.compile(optimizer='adam', loss='mse')
        es = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
        history = model_lstm.fit(X, y_lstm, epochs=10, batch_size=32, verbose=0, callbacks=[es])
        if history.history['loss'][-1] < best_loss:
            best_loss = history.history['loss'][-1]
            best_model = model_lstm
            best_model.save(f"{SAVE_DIR}/best_lstm_model_units_{units}_win_{abs(win)}.h5")

    last_window = res_scaled[-window_size:].reshape(1, window_size, 1)
    pred_res_scaled = best_model.predict(last_window)
    pred_res = scaler.inverse_transform(pred_res_scaled.reshape(-1, 1)).flatten()

    future = model.make_future_dataframe(periods=max(horizons), freq='min')
    forecast_future = model.predict(future).tail(max(horizons))
    yhat_prophet = forecast_future['yhat'].values
    y_true_all = df['y'].iloc[win:win + max(horizons)].values

    for h in horizons:
        y_pred_hybrid = yhat_prophet[:h] + pred_res[:h]
        y_true = y_true_all[:h]

        mae = mean_absolute_error(y_true, y_pred_hybrid)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred_hybrid))
        r2 = r2_score(y_true, y_pred_hybrid)
        mape = mean_absolute_percentage_error(y_true, y_pred_hybrid)
        accuracy = max(0, 100 * (1 - mape))

        results.append((abs(win), h, mae, rmse, r2, mape, accuracy))


In [ ]:
result_df = pd.DataFrame(results, columns=['Window (min ago)', 'Horizon (min)', 'MAE', 'RMSE', 'R2', 'MAPE', 'Accuracy (%)'])
result_df.to_csv(f"{SAVE_DIR}/hybrid_multiwindow_evaluation.csv", index=False)
result_df


In [ ]:
heatmap_data = result_df.pivot(index="Window (min ago)", columns="Horizon (min)", values="Accuracy (%)")

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlGnBu", cbar_kws={'label': 'Accuracy (%)'})
plt.title("Hybrid Forecasting Accuracy (%) by Window and Horizon")
plt.ylabel("Window (Minutes Ago)")
plt.xlabel("Forecast Horizon (Minutes Ahead)")
plt.tight_layout()
plt.savefig(f"{SAVE_DIR}/hybrid_accuracy_heatmap.png")
plt.show()
